In [1]:
%load_ext autoreload
%autoreload 2
from helpers import *
from NLP import *
from SentenceSplitterML import *
from TokenizerML import *
from lexicons import *
import nltk as nltk
from Parser import *
from grammar import *
import daft
import matplotlib.pyplot as plt
import networkx as nx

Loading and tokenizing sentences

In [ ]:
name = 'p2_data1'
text = get_text(name)
tokenizer = NLP(text)
tokenizer.sentence_split()
sentences = tokenizer.sentences
for sentence in sentences:
    t = NLP(sentence)
    t.tokenize()
    tokens = t.dirty_tokens
    print(nltk.pos_tag(tokens[0:-1]))

In [ ]:
nltk.download()

In [38]:
text = nltk.tokenize.word_tokenize("Get the flight through Huston")
tag = nltk.pos_tag(text)
print(tag)

[('Get', 'VB'), ('the', 'DT'), ('flight', 'NN'), ('through', 'IN'), ('Huston', 'NNP')]


In [40]:
text = "I bought a red gift for my friend yesterday"
text = "Get the flight through Huston"
#text = "The flight includes a meal"

p = Parser(text)
print(p.parsed_text)
table, links, my_table = p.parse()
p.save_table("data/parse_table.txt")
#p.make_tree(my_table)
print(links)
for tab in my_table:
    print(tab)

[('Get', 'VB'), ('the', 'DT'), ('flight', 'NN'), ('through', 'IN'), ('Huston', 'NNP')]
[['Det', 'Nominal'], ['Verb', 'NP'], ['Verb', 'NP'], ['Prep', 'NP'], ['Nominal', 'PP'], ['Det', 'Nominal'], ['Verb', 'NP'], ['Verb', 'NP'], ['VP', 'PP'], ['VP', 'PP']]
[[], [], [[(0, 0, 'Verb'), (1, 2, 'NP'), 'S'], [(0, 0, 'Verb'), (1, 2, 'NP'), 'VP']], [], [[(0, 0, 'Verb'), (1, 4, 'NP'), 'S'], [(0, 0, 'Verb'), (1, 4, 'NP'), 'VP'], [(0, 2, 'VP'), (3, 4, 'PP'), 'S'], [(0, 2, 'VP'), (3, 4, 'PP'), 'VP']]]
[[], [], [[(1, 1, 'Det'), (2, 2, 'Nominal'), 'NP']], [], [[(1, 1, 'Det'), (2, 4, 'Nominal'), 'NP']]]
[[], [], [], [], [[(2, 2, 'Nominal'), (3, 4, 'PP'), 'Nominal']]]
[[], [], [], [], [[(3, 3, 'Prep'), (4, 4, 'NP'), 'PP']]]
[[], [], [], [], []]


In [ ]:
G = nx.Graph()
for i in range(len(links)):
    if i == 0:
        G.add_node(links[i][0], label="Dinno", )
        G.add_edges_from([(links[i][0], links[i][1][0]), (links[i][0], links[i][1][1])])
    else:
        G.add_edges_from([(links[i][0], links[i][1][0]), (links[i][0], links[i][1][1])])
nx.draw(G, with_labels = True)


In [ ]:
L = 10
H = 10
pgm = daft.PGM(shape=[L, H])

y = H
x = L/2
pgm.add_node(daft.Node('0', links[0][0], L/2, H))
for i in range(len(links)):
    y = y - 1
    for j in range(len(links[i])):
        pgm.add_node(daft.Node(str(i)+'_'+str(j), links[i][1][j], x+j+i, y))
        if i == 0:
            pgm.add_edge('0', str(i)+'_'+str(j))
        else:
            pgm.add_edge(str(i-1)+'_'+str(1), str(i)+'_'+str(j))

pgm.render()
plt.show()